# Book Recommender System

### We'll use the clustering machine learning and collaborative 
### based for the recommendation

In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [40]:
data= pd.read_csv('BX_Books.csv', sep=";", error_bad_lines=False,encoding='latin-1')

C:\Users\hp\AppData\Local\Temp\ipykernel_3076\3062829561.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data= pd.read_csv('BX_Books.csv', sep=";", error_bad_lines=False,encoding='latin-1')


In [41]:
data.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton & Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [42]:
 data.shape

(271379, 8)

In [43]:
data.columns

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')

In [44]:
# we'll drop certain columns to remain with a few 

data= data[['ISBN','Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-L']]

In [45]:
data.head(1)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...


In [46]:
#we'll rename certain columns
data.rename(columns={
    "Book-Title" : "title",
    "Book-Author" : "author",
    "Year-Of-Publication":"year",
    "Publisher": "publisher",
    "Image-URL-L" : "img_url"}, inplace=True)

In [47]:
data.head(1)

,ISBN,title,author,year,publisher,img_url
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...


### Load the users dataset

In [48]:
users=  pd.read_csv('BX-Users.csv', sep=";", error_bad_lines=False,encoding='latin-1')

C:\Users\hp\AppData\Local\Temp\ipykernel_3076\41891608.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  users=  pd.read_csv('BX-Users.csv', sep=";", error_bad_lines=False,encoding='latin-1')


In [49]:
users.head(1)

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN


In [50]:
users.shape

(278858, 3)

### Load the ratings dataset

In [51]:
ratings=  pd.read_csv('BX-Book-Ratings.csv', sep=";", error_bad_lines=False,encoding='latin-1')

C:\Users\hp\AppData\Local\Temp\ipykernel_3076\3643727343.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  ratings=  pd.read_csv('BX-Book-Ratings.csv', sep=";", error_bad_lines=False,encoding='latin-1')


In [52]:
ratings.head(2)

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5


In [53]:
ratings.shape

(1149780, 3)

In [54]:
ratings.rename(columns={
    "User-ID": "user_id",
    "Book-Rating": "rating"},inplace=True)

In [55]:
ratings.head(2)

,user_id,ISBN,rating
0,276725,034545104X,0
1,276726,0155061224,5


### compare the 3 datasets

User_id 276725 rated book 034545104X a 5 

In Collaborative recommendation type the ratings are mostly used.

In [56]:
print(data.shape)
print(users.shape)
print(ratings.shape)

(271379, 6)
(278858, 3)
(1149780, 3)


In [57]:
#to check the number of ratings each user has made

ratings['user_id'].value_counts()

11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
116180        1
116166        1
116154        1
116137        1
276723        1
Name: user_id, Length: 105283, dtype: int64

User_id 11676 has rated 13602 books

#### we'll sample the number of user_id who have rated more than 200 times

In [58]:
x = ratings['user_id'].value_counts() > 200

In [59]:
x[x].shape

(899,)

#### out of the 105283 users, 899 have rated over 200 books

In [60]:
#convert the user_ids to indexes

y=x[x].index

In [61]:
y

Int64Index([ 11676, 198711, 153662,  98391,  35859, 212898, 278418,  76352,
            110973, 235105,
            ...
            260183,  73681,  44296, 155916,   9856, 274808,  28634,  59727,
            268622, 188951],
           dtype='int64', length=899)

In [62]:
#filter the user_id that have rated more than 200 times

rating = ratings[ratings['user_id'].isin(y)]

In [63]:
rating.head(3)

,user_id,ISBN,rating
1456,277427,002542730X,10
1457,277427,0026217457,0
1458,277427,003008685X,8


#### Now we'll combine data(the list of all books) with the list of user_id 
that have rated over 200 times, to get the list of  the books 

In [64]:
data.head(3)

,ISBN,title,author,year,publisher,img_url
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...


### Since the primary key for merging is ISBN we'll use it.

### It performs an inner join to the datasets

In [65]:
ratings_with_books= ratings.merge(data, on ="ISBN")

In [66]:
ratings_with_books.head(3)

,user_id,ISBN,rating,title,author,year,publisher,img_url
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...
2,6543,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...


In [67]:
ratings_with_books.shape

(1031175, 8)

#### We've gotten the books with the user_id that have been rated more than 200 times

In [71]:
#to get the count of ratings for each book not the total sum of ratings then  grouped by the title
num_rating =ratings_with_books.groupby('title')['rating'].count().reset_index()

In [72]:
num_rating.head(3)

,title,rating
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1


In [73]:
#we'll change the ratings to num_of_rating
num_rating.rename(columns={'rating': 'num_of_rating'}, inplace=True)

#### we'll look for the ratings for each and every book

In [74]:
num_rating.head()

,title,num_of_rating
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1


In [75]:
ratings_with_books.head(3)

,user_id,ISBN,rating,title,author,year,publisher,img_url
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...
2,6543,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...


#### we'll merge the number of ratings per book to the ratings_with_books using title

In [76]:
final_rating = ratings_with_books.merge(num_rating, on='title')

In [77]:
final_rating.head(3)

,user_id,ISBN,rating,title,author,year,publisher,img_url,num_of_rating
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,60
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,60
2,6543,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,60


#### now we'll go onto search the books that have got more than 50 ratings

In [108]:
final_rating=final_rating[final_rating['num_of_rating']>=50]

In [109]:
final_rating.tail(2)

,user_id,ISBN,rating,title,author,year,publisher,img_url,num_of_rating
730571,244349,0061092096,0,Love in Another Town,Barbara Taylor Bradford,1996,HarperTorch,http://images.amazon.com/images/P/0061092096.0...,68
730572,258352,0061092096,0,Love in Another Town,Barbara Taylor Bradford,1996,HarperTorch,http://images.amazon.com/images/P/0061092096.0...,68


In [110]:
final_rating.shape

(285518, 9)

#### we'll drop columns with duplicate user_id and title

In [111]:
final_rating.drop_duplicates(['user_id','title'], inplace=True)

In [112]:
final_rating.shape

(285518, 9)

#### turn dataset into pivots 

In [113]:
book_pivot = final_rating.pivot_table(columns='user_id', index='title', values='rating')

In [114]:
book_pivot.head(4)

user_id,8,9,14,16,17,19,23,26,32,39,...,278820,278824,278828,278832,278836,278843,278844,278846,278851,278854
title,,,,,,,,,,,,,,,,,,,,,
10 Lb. Penalty,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16 Lighthouse Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1984,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1st to Die: A Novel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


This shows the user_id ratings for each book

In [118]:
book_pivot.shape

(2444, 47994)

In [119]:
book_pivot.fillna(0, inplace=True)

In [120]:
book_pivot

user_id,8,9,14,16,17,19,23,26,32,39,...,278820,278824,278828,278832,278836,278843,278844,278846,278851,278854
title,,,,,,,,,,,,,,,,,,,,,
10 Lb. Penalty,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16 Lighthouse Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1984,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010: Odyssey Two,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zoya,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"\O\"" Is for Outlaw""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### The zeros will bring about problems when clustering, we'll use 0 to compress the rows

In [121]:
from scipy.sparse import csr_matrix

In [122]:
book_sparse = csr_matrix(book_pivot)

In [123]:
book_sparse

<2444x47994 sparse matrix of type '<class 'numpy.float64'>'
	with 113444 stored elements in Compressed Sparse Row format>

# Fitting the Model

In [124]:
from sklearn.neighbors import NearestNeighbors


In [125]:
# select the model and algorithm used for clustering

model = NearestNeighbors(algorithm='brute')

In [126]:
model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

In [127]:
distance,suggestion = model.kneighbors(book_pivot.iloc[237, :].values.reshape(1,-1), n_neighbors=6)

In [128]:
distance


array([[ 0.        , 73.70888685, 74.13501197, 74.14849965, 74.25631286,
        74.42445835]])

In [129]:
suggestion

array([[ 237, 1391, 1713,  770,   65,  284]], dtype=int64)

In [130]:
for i in range (len(suggestion)):
    print(book_pivot.index[suggestion[i]])

Index(['Beloved (Plume Contemporary Fiction)', 'Reasonable Doubt',
       'The Blooding', 'Ground Zero and Beyond', 'A Secret Affair',
       'Blood and Gold (Rice, Anne, Vampire Chronicles.)'],
      dtype='object', name='title')


#### Storing all the books in a variable called books_name

In [131]:


books_name = (book_pivot.index)

#### storing the model, books_name, final_rating

In [132]:
#we've created a folder called artifacts to store the pickle files

import os
import pickle

# Create the 'artifacts' directory if it doesn't exist
os.makedirs('artifacts', exist_ok=True)

In [137]:
import pickle
pickle.dump(model, open('artifacts/model.pkl' , 'wb'))
pickle.dump(books_name, open('artifacts/books_name.pkl','wb'))
pickle.dump(final_rating, open('artifacts/final_rating.pkl','wb'))
pickle.dump(book_pivot, open('artifacts/books_pivot.pkl','wb'))



## Creating a function of how the application will be functioning

In [151]:
def recommend_book(book_name):
    book_id = np.where(book_pivot.index == book_name)[0][0]
    distance, suggestion = model.kneighbors(book_pivot.iloc[book_id, :].values.reshape(1,-1), n_neighbors=6)
    
    for i in range(len(suggestion)):
        books = book_pivot.index[suggestion[i]]
        for j in books:
            print(j)

In [152]:
book_name= 'Reasonable Doubt'
recommend_book(book_name)

Reasonable Doubt
The Blooding
Ground Zero and Beyond
A Secret Affair
Blood and Gold (Rice, Anne, Vampire Chronicles.)
Women in His Life
